In [ ]:
from transformato import load_config_yaml, SystemStructure, IntermediateStateFactory, ProposeMutationRoute
from transformato.mutate import perform_mutations
from IPython.display import SVG
from transformato.utils import run_simulation, postprocessing
import warnings
warnings.filterwarnings("ignore", module='parmed')


## Load the yaml configuration files

In [ ]:
configuration = load_config_yaml(config='toluene-methane-rsfe.yaml',
                       input_dir='../data/', output_dir='.')

## Let's look at some of the parameters that are defined for this transformation. This is a test case, so we run a very short sampling phase of 100,000 steps with 0.1fs dt. 

In [ ]:
configuration['simulation']

## Generate structures as defined in the yaml config files and generate the mutation factory.

In [ ]:
s1 = SystemStructure(configuration, "structure1")
s2 = SystemStructure(configuration, "structure2")
s1_to_s2 = ProposeMutationRoute(s1, s2)

## Propose a route from structure1 and structure2 to a commen core that has the same elements, but may differ in atom types, bonded parameters and charges. The commen core is highlighted in red on both structures.

In [ ]:
s1_to_s2.propose_common_core()

## The common core is accepted without modificatinos and the necessary steps to reach the common core are printed

In [ ]:
s1_to_s2.finish_common_core()

## Show the common core with atom types

In [ ]:
SVG(s1_to_s2.show_common_core_on_mol1())

In [ ]:
SVG(s1_to_s2.show_common_core_on_mol2())

## Generate the mutation list that is necessary to transform structure1 to the common core.
## The intermediate states that are generated are located in different directories and can be run independently.

In [ ]:
# generate the mutation list for the original
mutation_list = s1_to_s2.generate_mutations_to_common_core_for_mol1()
print(mutation_list.keys())
i = IntermediateStateFactory(
system=s1,
configuration=configuration,
)

## Now the topologies for the alchemical states are generated. If you have a non-trivial mutation (i.e. a mutation for which the vdw potential of more than a single heavy atom will be turned-off) you should define the order in which the heavy atoms are turned off in `list_of_heavy_atoms_to_be_mutated`. It is possible to define more than a single atom to be turned off for an alchemical state by using tuple (i.e. `(11,9)`).

In [ ]:
perform_mutations(configuration=configuration, 
i=i, 
mutation_list=mutation_list, 
list_of_heavy_atoms_to_be_mutated=[13, (11,9), (3,1)])

## Congratulations! You have generated all intermediate states to generate equilibrium samples from distributions that alchemically connect the one endpoint to the common core.
## No we need to start the sampling phase

In [ ]:
run_simulation(i.output_files, engine='openMM')

## Now let's calculate the free energy of ethane to the common core

In [ ]:
ddG_openMM, dddG, f_openMM = postprocessing(
    configuration,
    name="toluene",
    engine="openMM",
    max_snapshots=600,
)
print(f'Free energy difference: {ddG_openMM} +- {dddG} [kT')


In [ ]:
mutation_list = s1_to_s2.generate_mutations_to_common_core_for_mol2()
print(mutation_list.keys())
i = IntermediateStateFactory(
system=s2,
configuration=configuration,
)
perform_mutations(configuration=configuration, i=i, mutation_list=mutation_list, nr_of_mutation_steps_charge=1)
run_simulation(i.output_files, engine='openMM')
ddG_openMM, dddG, f_openMM = postprocessing(
    configuration,
    name="methane",
    engine="openMM",
    max_snapshots=600,
)
print(f'Free energy difference: {ddG_openMM} +- {dddG} [kT')